# Step 2: Using pandas and regex to clean the alerts

With all the alerts I've scraped, now is the time to clean our data. 

In [2]:
import re

In [3]:
import pandas as pd
df = pd.read_csv('november-delays.csv')

In [4]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
df.head(2)

,date,agency,subject,message
0,11/30/22 10:59 PM,NYC,"Update: BX, 2 Train, Delays",Northbound 2 trains are running with delays after we investigated why a train's brakes were activated at Allerton Av.
1,11/30/22 10:46 PM,NYC,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.


## Getting rid of unwanted rows

In [5]:
# Getting rid of unrelated rows
df = df[df['subject'].str.contains('Menu')==False]
df = df[df['subject'].str.contains('pages')==False]

# Getting rid of rows that start with "Update:" 
# as those rows are updates on alerts that were posted already.
df = df[df['subject'].str.contains('Update')==False]

# Getting rid of alerts:
# planned maintenance, holiday schedule announcement, and service resume alerts
df = df[df['subject'].str.contains('Planned', case=False)==False]
df = df[df['subject'].str.contains('Holiday')==False]
df = df[df['subject'].str.contains('resumed')==False]

# Getting rid of interruptions caused by marine traffic
df = df[df['message'].str.contains('marine traffic')==False]

# Gettimg rid of interruptions caused by palnned maintenance
df = df[df['message'].str.contains('scheduled|planned')==False]

# reset index
df.head()

,date,agency,subject,message
1,11/30/22 10:46 PM,NYC,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.
3,11/30/22 10:37 PM,NYC,"BKLYN, D and Q Trains, Delays",Northbound D/Q trains are delayed while we address a mechanical problem on a train near DeKalb Av.
5,11/30/22 10:13 PM,NYC,"BX, 6 Train, Delays",Northbound 6 trains are running with delays after we corrected a switch malfunction near Buhre Av.
7,11/30/22 10:06 PM,NYC,"MANH, D Train, Delays",Northbound D trains are delayed while we investigate why a train's brakes were activated at Broadway-Lafayette St.
9,11/30/22 9:37 PM,NYC,"BKLYN, D and N Trains, Delays",Northbound D/N trains are delayed while we investigate why a train's brakes activated at 36 St.


## Extract stop information from the message column

In [6]:
stop_regex = r"\b(?:at|near|leaving|in the) ([^.\n]+)"
df['station'] = df.message.str.extract(stop_regex, expand=False)
df['station'] = df['station'].fillna(df.message.str.extract(r"\b(?:between) ([^.\n]+)", expand=False))
df.head(2)

,date,agency,subject,message,station
1,11/30/22 10:46 PM,NYC,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.,Allerton Av
3,11/30/22 10:37 PM,NYC,"BKLYN, D and Q Trains, Delays",Northbound D/Q trains are delayed while we address a mechanical problem on a train near DeKalb Av.,DeKalb Av


In [7]:
df.station.value_counts()

125 St                         34
14 St-Union Sq                 30
Atlantic Av-Barclays Ctr       28
59 St-Columbus Circle          28
Times Sq-42 St                 25
                               ..
39 Av-Dutch Kills               1
Jay St-MetroTech                1
Longwood Av                     1
Chambers St and Franklin St     1
80 St                           1
Name: station, Length: 398, dtype: int64

## Extract cause of interruption from the message column

In [8]:
cause_regex = r"(brake*|signal|switch|door|mechanical problem|\
fire|EMS|track maintenance|disruptive|smoke|investigation|\
cleaning|person struck|vandali\w*|tree|medical help|\
remove debris|unauthorized person|fallen leaves|replace rails|\
remove a train|struck by a train|rail|customer altercation|\
FDNY assistance|low-hanging wires|debris|loss\s*of\s*\w*\s*power|\
water|track work|people on the tracks|person in need|garbage|\
trash|riding on|someone in need|communication|maintenance|\
We're running as much service as we can)"
df['cause'] = df.message.str.extract(cause_regex, expand=False)

df.head(2)

,date,agency,subject,message,station,cause
1,11/30/22 10:46 PM,NYC,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.,Allerton Av,brake
3,11/30/22 10:37 PM,NYC,"BKLYN, D and Q Trains, Delays",Northbound D/Q trains are delayed while we address a mechanical problem on a train near DeKalb Av.,DeKalb Av,mechanical problem


In [170]:
df[df.cause.isna()].head()

,date,agency,subject,message,station,cause
269,11/29/22 5:38 AM,NYC,"J Train, Delays",You may wait longer for a northbound J train. We are running as much service as we can with the crews we have available.,NaN,NaN
379,11/28/22 11:21 AM,NYC,"BKLYN, MANH, QNS, A, C, and F Trains, Multiple Changes",Some northbound F via the A/C line from Jay St-MetroTech to W 4 St-Wash Sq. Some northbound F via the G line from Bergen St to Queens Plaza. Some Forest Hills-71 Av-bound M trains will end at Chambers St on the J line. Expect a longer wait for northbound A/C trains while they share the track with rerouted trains.,Chambers St on the J line,NaN
417,11/28/22 12:49 AM,NYC,"BKLYN, MANH, QNS, E and F Trains, Delays",World Trade Center-bound E and Coney Island-Stillwell Av-bound F trains are delayed while we move a train into storage from Forest Hills-71 Av.,NaN,NaN
460,11/27/22 10:24 AM,NYC,"BKLYN, Franklin Av S Train, Delays",Southbound FS trains are delayed while we request NYPD assistance to remove a dog from the tracks at Botanic Garden.,Botanic Garden,NaN
495,11/26/22 10:24 PM,NYC,"SIR Train, Boarding Change","On Sunday, November 27th from 7AM-7PM SIR trains from Jefferson Av to Old Town will board at the Tottenville-bound platform. Boarding change included Jefferson Av, Dongan Hills, and Old Town stations. What's Happening? Scheduled Maintenance",the Tottenville-bound platform,NaN


In [10]:
df[df.message.str.contains('dog')]

,date,agency,subject,message,station,cause
460,11/27/22 10:24 AM,NYC,"BKLYN, Franklin Av S Train, Delays",Southbound FS trains are delayed while we request NYPD assistance to remove a dog from the tracks at Botanic Garden.,Botanic Garden,NaN


In [171]:
df.cause.value_counts()

brake                                      246
disruptive                                 185
EMS                                        163
signal                                     128
remove a train                             118
mechanical problem                         114
door                                        82
switch                                      65
investigation                               54
track maintenance                           49
unauthorized person                         41
rail                                        36
vandalized                                  28
fire                                        27
medical help                                17
We're running as much service as we can     14
struck by a train                           12
cleaning                                    11
smoke                                       11
replace rails                               10
remove debris                                9
debris       

In [172]:
df.to_csv("uncleaned-delays.csv",index=False)

## Manually cleaning the columns to double check mistakes after regex 
### Things that were fixed manually:
station column:
* change the station of rows that did not specify location to "na";
* deleting unrelated phrases that come after the name of station in station column;
* Fixed spelling mistakes, wrong spacing.

cause column:

* Changing some rows where message contains "investigation" but did not specify the problem to "na";
* Changing the cause of rows with message containing "loss of third-rail power" from "rail" to "loss of power;"
* Specify cause for "remove a train" cause column;
* Specify cause for rows with non-value in cause column, unspecified cuase are marked as "na."


### I manually cleaned ```uncleaned-delays.csv``` using Excel and saved to ```manually-cleaned-delays.csv```

In [173]:
# Read in the manually cleaned file
cleaned_df = pd.read_csv("manually-cleaned-delays.csv")

In [174]:
# Getting rid of message than contains "resumed" and "scheduled maintenance,"
# as these rows are updates on trains resuming services after delays, or planned maintenance.
cleaned_df = cleaned_df[cleaned_df['message'].str.contains('resume*|scheduled|planned', case = False)==False]
cleaned_df.head()

,index,date,agency,subject,message,station,cause
0,1,11/30/22 22:46,NYC,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.,Allerton Av,brake
1,3,11/30/22 22:37,NYC,"BKLYN, D and Q Trains, Delays",Northbound D/Q trains are delayed while we address a mechanical problem on a train near DeKalb Av.,DeKalb Av,mechanical problem
2,5,11/30/22 22:13,NYC,"BX, 6 Train, Delays",Northbound 6 trains are running with delays after we corrected a switch malfunction near Buhre Av.,Buhre Av,switch
3,7,11/30/22 22:06,NYC,"MANH, D Train, Delays",Northbound D trains are delayed while we investigate why a train's brakes were activated at Broadway-Lafayette St.,Broadway-Lafayette St,brake
4,9,11/30/22 21:37,NYC,"BKLYN, D and N Trains, Delays",Northbound D/N trains are delayed while we investigate why a train's brakes activated at 36 St.,36 St,brake


In [175]:
# Checking each cause category
cleaned_df.cause.value_counts()

brake                                      235
disruptive                                 186
mechanical problem                         156
EMS                                        154
signal                                     118
door                                       102
switch                                      58
track maintenance                           45
cleaning                                    44
unauthorized person                         42
investigation                               41
vandalized                                  28
na                                          26
fire                                        21
rail                                        17
loss of power                               16
medical help                                16
We're running as much service as we can     14
smoke                                       11
replace rails                               10
struck by a train                           10
remove debris

In [176]:
cleaned_df[cleaned_df.cause == 'low-hanging wires']

,index,date,agency,subject,message,station,cause
256,531,11/26/22 9:35,NYC,"BKLYN, L Train, Part Suspended","There is no L service between Lorimer St and Myrtle-Wyckoff Avs while our crews secure low-hanging wires over the tracks near Graham Av. As an alternative, consider taking local B24, B57, B60 or Q54 buses to/from nearby G train stations. For service between DeKalb Av and Myrtle-Wyckoff Avs, consider using the B13 bus.",Graham Av,low-hanging wires
257,535,11/26/22 8:55,NYC,"BKLYN, L Train, Part Suspended",There is no L service between Lorimer St and Myrtle-Wyckoff Avs while we our crews secure low-hanging wires over the tracks near Graham Av.,Graham Av,low-hanging wires


In [177]:
# Dropping unwanted columns
cleaned_df = cleaned_df.drop(['index', 'agency'], axis=1)
cleaned_df.head()

,date,subject,message,station,cause
0,11/30/22 22:46,"BX, 2 Train, Delays",Northbound 2 trains are delayed while we investigate why a train's brakes were activated at Allerton Av.,Allerton Av,brake
1,11/30/22 22:37,"BKLYN, D and Q Trains, Delays",Northbound D/Q trains are delayed while we address a mechanical problem on a train near DeKalb Av.,DeKalb Av,mechanical problem
2,11/30/22 22:13,"BX, 6 Train, Delays",Northbound 6 trains are running with delays after we corrected a switch malfunction near Buhre Av.,Buhre Av,switch
3,11/30/22 22:06,"MANH, D Train, Delays",Northbound D trains are delayed while we investigate why a train's brakes were activated at Broadway-Lafayette St.,Broadway-Lafayette St,brake
4,11/30/22 21:37,"BKLYN, D and N Trains, Delays",Northbound D/N trains are delayed while we investigate why a train's brakes activated at 36 St.,36 St,brake


In [179]:
# save dataframe to a new .csv file
cleaned_df.to_csv("cleaned-delays.csv", index=False)

For data processing and analysis, please see notebook *```Step 03 - Processed for final dataframe```*.